In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.3.0


# Get dataset

In [2]:
# use MNIST as save load example, use only 1000 samples for efficiency
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0 # convert to 1D and normalize
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# Create model

In [3]:
def create_model():
    model = tf.keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Save weights

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# create a callback that saves model weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

# train the model while saving checkpoints
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[cp_callback])

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2143 - sparse_categorical_accuracy: 0.6496
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 1.1608 - sparse_categorical_accuracy: 0.6660 - val_loss: 0.7124 - val_sparse_categorical_accuracy: 0.7770
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4218 - sparse_categorical_accuracy: 0.8873
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.4150 - sparse_categorical_accuracy: 0.8900 - val_loss: 0.5249 - val_sparse_categorical_accuracy: 0.8430
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2886 - sparse_categorical_accuracy: 0.9241
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2832 - sparse_categorical_accuracy: 0.9260 - val_loss: 0.4798 - val_sparse_categorical_accuracy: 0.8530
Epoch 4/10
29/3

In [5]:
ls {checkpoint_dir}

Volume in drive C has no label.
 Volume Serial Number is 46A9-68DD

 Directory of c:\Users\kevin\Documents\PythonProjects\TensorflowCoding\Basics\training_1

09/22/2020  01:14 PM    <DIR>          .
09/22/2020  01:14 PM    <DIR>          ..
09/22/2020  01:14 PM                71 checkpoint
09/22/2020  01:14 PM         4,886,673 cp.ckpt.data-00000-of-00001
09/22/2020  01:14 PM             1,222 cp.ckpt.index
               3 File(s)      4,887,966 bytes
               2 Dir(s)   7,136,460,800 bytes free


# Load checkpoint

In [6]:
# create basic model without training
model = create_model()

# evaluate model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4614 - sparse_categorical_accuracy: 0.0370
Untrained model, accuracy:  3.70%


In [7]:
# load the weights
model.load_weights(checkpoint_path)

# evaluate model with loaded weights
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3932 - sparse_categorical_accuracy: 0.8700
Restored model, accuracy: 87.00%


# Checkpoint configuration

In [8]:
# include epoch in the file name
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# create callback that saves models every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

model = create_model()

# save weights using the checkpoint_path format
model.save_weights(checkpoint_path.format(epoch=0))

# train the model while saving checkpoints
model.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels), callbacks=[cp_callback], verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [10]:
# evaluate the latest weight

model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4949 - sparse_categorical_accuracy: 0.8720
Restored model, accuracy: 87.20%


# Manually save weights

In [11]:
# can also manually save weights, e.g. at the end of training
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

# restore weights
model.load_weights('./checkpoints/my_checkpoint')

# eval restored weights
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4949 - sparse_categorical_accuracy: 0.8720
Restored model, accuracy: 87.20%


# Save entire model

In [12]:
# can save entire model including architecture, weights, training configurations, and then can be exported
# optimizer-state is also saved and recovered, so can continue training when loaded
# can be save in either SavedModel or HDF5 format, and can be loaded in TensorFlow.js or TensorFlow Lite

### SavedModel

In [13]:
# create and train a new model
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# save entire model as SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1427 - sparse_categorical_accuracy: 0.6660
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4257 - sparse_categorical_accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2793 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1962 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1566 - sparse_categorical_accuracy: 0.9640
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [15]:
# reload the SavedModel
new_model = tf.keras.models.load_model('saved_model/my_model')

# check reload architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# evaluate restored model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4125 - sparse_categorical_accuracy: 0.8700
Restored model, accuracy: 87.00%


### HDF5

In [17]:
# create and train a new model
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# save entire model as HDF5
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1270 - sparse_categorical_accuracy: 0.6860
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4314 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2691 - sparse_categorical_accuracy: 0.9340
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2080 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1490 - sparse_categorical_accuracy: 0.9650


In [18]:
# reload HDF5 model
new_model = tf.keras.models.load_model('my_model.h5')

# check reload architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
# evaluate restored model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4623 - sparse_categorical_accuracy: 0.8550
Restored model, accuracy: 85.50%
